**Prompt Optimization & Experiment Tracking with MLflow**

This tutorial demonstrates how to optimize prompts and track experiments using MLflow through a practical example.
The Task: Finding Melvin

We'll train an AI to identify Melvin, a specific cat, from various cat photos. The AI must:

    ✅ Accept images containing Melvin
    ❌ Reject images without Melvin

While humans find this task trivial, AI systems struggle because they lack our common knowledge assumptions about visual recognition.
Why Use MLflow?

Traditional approaches (classical computer vision, machine learning) would require significant time and resources. Using AI is more efficient, but prompt optimization requires systematic experimentation.

MLflow helps us track what works by recording inputs and ouputs for each experiment and providing a convenient UI for results analysis. 

How MLflow Works

    Set storage location - Local folder or cloud storage
    Define tracking parameters - Choose what to monitor
    Run experiments - MLflow creates subfolders for each run
    View results - Launch GUI with poetry run mlflow ui

In [ ]:
# Imports and system configuration
%load_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
import mlflow

from prompt_engineering.utils import CatIdentifier, evaluate_and_plot # custom code written for this experiment. 

image_path = Path("../data/images/") # images are stored locally for this experiment
cat_identifier = CatIdentifier("openai/gpt-5-mini") # initialize and instance of our cat identifier using a decent multi-modal model
label_df = pd.read_csv("../data/labels.csv") # manualy generated (by me) labels are stored locally

mlflow.set_tracking_uri("../mlruns") # this determines where our ml run data is stored. We're using a local folder here.
mlflow.set_experiment("cat_id_prompt_optimization") # set an experiment name to anything descriptive and appropriate.

# Experimentation

## System Prompt V1

For the first pass, lets use a simple prompt

In [ ]:
system_prompt_v1 = """
    Your user owns a tabby cat named Melvin. Your job is to identify whether the image provided is of Melvin, or some other cat.
    Respond with your decision, confidence, and reasoning.
    """

prompt_description_v1 = """
    Baseline
    """

This prompt is certain to fail. All it tells us is that the cat is a tabby, which represents the mojority of the domesticated cat population. We clearly need to do better but lets use this as a baseline

In [ ]:
with mlflow.start_run(run_name = "Baseline performance (notebook)"): # start_run tells mlflow that we're about to conduct a new experiment. We can name this experiment.
    mlflow.log_param("prompt_version", "v1") # log that our "prompt version" is  "v1" 
    mlflow.log_param("prompt_description", prompt_description_v1) # optionally log a description of the prompt.

    # for each cat image, generate a prediction from the AI
    results= {image_id: cat_identifier.identify(f"{image_path}/{image_id}.jpg", system_prompt_v1) for image_id in label_df.image_id[:2]}
    preds = {k:v["cat"] for k,v in results.items()}

    acc = evaluate_and_plot(preds, label_df, image_path) # evaluate the predictions against the labels

    mlflow.log_metric("Accuracy", acc) # log the accuracy using mlflow

In [ ]:
print(results) # take a look at the reasoning given for each prediciton - this can help us tune future prompts.

## System Prompt V2

Our previous prompt described only that Melvin is a tabby, which is not enough to distinguish him from other tabby cats. We need to provide more details to help the model identify the specific cat we are referring to. Lets try and improve our verbal description. 


In [ ]:
system_prompt_v2 = f"""
    Your user owns a tabby cat named Melvin. Your job is to identify whether the image provided is of Melvin, or some other cat.
    Use the following description of Melvin to help you make your decision:
    Melvin:
        1. Bicolor tabby coat with white underside — Distinctive contrast of dark brown/gray mackerel tabby striping on the back and flanks paired with a large, solid white chest and belly; location: back/sides (tabby), chest/underside (white).

        2. Narrow white blaze up the center of the face — A thin vertical white stripe running from the upper muzzle between the eyes toward the forehead, bordered by darker tabby patches on either side; location: center of the face (muzzle to forehead).

        3. Pink nose centered in a white muzzle — Prominent pink nose set into a clean white whisker pad and chin area, creating a clear facial focal point; location: nose and lower face/muzzle.
       
        4. Large round light green eyes with dark rims — Bold, widely set round eyes of light green color framed by darker tabby markings that accentuate their shape; location: eyes/eye rims.

        5. Ringed dark tail and white paws — Tail appears uniformly dark with subtle ringed banding toward the base and all four paws predominantly white; location: tail and feet.

    When making your descision, consider whether the location of a described feature is visible but that feature is clearly absent. 
    If you aren't certain, or any feature is missing, respond with "other".
    Respond with your decision, confidence, and reasoning.
    """
prompt_description_v2 = """
    A feature-based description of Melvin is used to help the AI make a distinction. 
    """

In [ ]:
# This cell is the same as before, but we've updated to use the new v2 prompt
with mlflow.start_run(run_name = "Distinctive description (notebook)"):
    mlflow.log_param("prompt_version", "v2")
    mlflow.log_param("promt_description", prompt_description_v2)

    results = {image_id: cat_identifier.identify(f"{image_path}/{image_id}.jpg", system_prompt_v2) for image_id in label_df.image_id}
    preds = {k:v["cat"] for k,v in results.items()}

    acc = evaluate_and_plot(preds, label_df, image_path)

    mlflow.log_metric("Accuracy", acc) 

## System Prompt V3

The v2 prompt is already quite good, but if you run it repeatedly, you'll see that it can flip-flop and make mistakes. We can improve it by instructing the AI to follow a chain-of-thought review process and to recursively reconsider its conclusions.
This is kind of like spelling out to the AI how it should think, a skill we usually take for granted.

In [ ]:
system_prompt_v3 = f"""
    Your user owns a tabby cat named Melvin. Your job is to identify whether the image provided is of Melvin, or some other cat.
    Step 1: Feature Analysis
    For each image, systematically check if their distinctive features are present.

    Melvin Features:
        1. Bicolor tabby coat with white underside — Distinctive contrast of dark brown/gray mackerel tabby striping on the back and flanks paired with a large, solid white chest and belly; location: back/sides (tabby), chest/underside (white).

        2. Narrow white blaze up the center of the face — A thin vertical white stripe running from the upper muzzle between the eyes toward the forehead, bordered by darker tabby patches on either side; location: center of the face (muzzle to forehead).

        3. Pink nose centered in a white muzzle — Prominent pink nose set into a clean white whisker pad and chin area, creating a clear facial focal point; location: nose and lower face/muzzle.

        4. Large round light green eyes with dark rims — Bold, widely set round eyes of light green color framed by darker tabby markings that accentuate their shape; location: eyes/eye rims.

        5. Ringed dark tail and white paws — Tail appears uniformly dark with subtle ringed banding toward the base and all four paws predominantly white; location: tail and feet.
    
    Step 2: For each feature, answer the following questions:
        a) Is the feature clearly and unambiguously present? (yes/no)
        b) If you were to describe the pictured cat, would you use the same language as in the provided description of Melvin?
        c) What is my confidence level that the pictured cat is an obvious match for the described feature

    Step 3: Consider the possibility that this cat is not Melvin. Does that explain the observations you made during feature analysis? 
    Re-examine your responses and your confidence levels from step 2. 
        
    Step 4: Final answer with confidence and reasoning. If you aren't certain, or any feature is missing, respond with "other". 
    """
prompt_description_v3 = """
    AI prompt enhanced with chain of thought reasoning and recursion instructions.
    """

In [ ]:
with mlflow.start_run(run_name = "Reasoning-enhanced prompt (notebook)"):
    mlflow.log_param("prompt_version", "v3")
    mlflow.log_param("run_description", prompt_description_v3)

    results = {image_id: cat_identifier.identify(f"{image_path}/{image_id}.jpg", system_prompt_v3) for image_id in label_df.image_id}
    preds = {k:v["cat"] for k,v in results.items()}

    acc = evaluate_and_plot(preds, label_df, image_path)

    mlflow.log_metric("Accuracy", acc) 

## System prompt v4

All this time, we've been attempting to optimize a prompt by manipulating the text we provide to the model, but this is an inefficient medium for a model which can process images just as easily as text. The smarter approach is therefore to provide a sample image of Melvin alongside the test image to the model can compare across the same modality.

In [ ]:
system_prompt_v4 = f"""
    Your user owns a tabby cat named Melvin. They will provide you with two images, a sample image of Melvin, and a test image. Your job is to determine whether the 
    test image is also Melvin, or some other cat.
    """
prompt_description_v4 = """
    Prompt with sample image 
    """

In [ ]:
with mlflow.start_run(run_name = "Sample image (notebook)"):
    mlflow.log_param("prompt_version", "v4")
    mlflow.log_param("run_description", prompt_description_v4)

    # DIFFERENCE: note the use of a identify_comp rather than identify. Identify comp takes a path to a sample image as well as a path to a test image.
    results = {image_id: cat_identifier.identify_comp("../data/sample_images/sample.jpg", f"{image_path}/{image_id}.jpg", system_prompt_v4) for image_id in label_df.image_id}
    preds = {k:v["cat"] for k,v in results.items()}

    acc = evaluate_and_plot(preds, label_df, image_path)

    mlflow.log_metric("Accuracy", acc) 

# Review

To review the experiment results, you can run 'mlflow ui' from the terminal (or 'poetry run mlflow ui' in our case). This will give you a web-based UI to explore the results of the experiments we've conducted above.

This notebook was meant to walk you through the basics of prompt engineering using MLflow. I highly recommend you take a look a the parallel path of using hydra for configuration management alongside MLflow. Hydra lets you consolodate your configuration changes to a few yaml files, making switching between configurations and tracking changes easy and efficient.